In [1]:
!pip install pytorch-pretrained-bert
! pip install nltk

     |████████████████████████████████| 133kB 4.8MB/s 


In [3]:
from google.colab import files
uploaded = files.upload()

Saving new_bow.txt to new_bow.txt
Saving summary2.txt to summary2.txt


In [4]:
#for importing csv file to which thw similarity values are going to be stored
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
%matplotlib
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from pytorch_pretrained_bert import BertTokenizer,BertModel,BertForMaskedLM
import logging

from sklearn.metrics.pairwise import cosine_similarity

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Using matplotlib backend: agg
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
file1 = "summary2.txt"
data1 = uploaded[file1].decode("utf-8").split("\r\n")

for i in range(len(data1)):
  data1[i] = data1[i].split(",")

In [0]:
file3 = "new_bow.txt"
data3 = uploaded[file3].decode("utf-8").split("\r\n")

for i in range(len(data3)):
  data3[i] = data3[i].split(",")

In [0]:
def Average(lst): 
    return sum(lst) / len(lst)

In [22]:
list4 = []
#loop for the tweet.txt files containing 10 lines having 15 tokens each.
for i in data1:
  for j in i:    
    list3 = []
    #input formatting
    marked_text = "[CLS]"+j+"[SEP]"
    #tokenization
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    #printing the tokens with their indices
    for tup in zip(tokenized_text,indexed_tokens):
      print('{:<12} {:>6}'.format(tup[0],tup[1]))
    segment_ids = [1] * len(tokenized_text)
    #converting python list to tensors as we are using pytorch and tensorflow code
    token_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segment_ids])
    model = BertModel.from_pretrained('bert-base-uncased')
    model.eval()
    #Hidden states
    with torch.no_grad():
      encoded_layers, _ = model(token_tensor,segments_tensors)
    print ("Number of layers:", len(encoded_layers))
    layer_i = 0

    print ("Number of batches:", len(encoded_layers[layer_i]))
    batch_i = 0

    print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
    token_i = 0

    print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

    token_i = 5
    layer_i = 5
    vec = encoded_layers[layer_i][batch_i][token_i]
    #
    token_vecs = encoded_layers[11][0]

    # Calculate the average of all token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)

    #loop for the bag of words
    for v in data3:
      for u in v:
        marked_text2 = "[CLS]"+u+"[SEP]"
        tokenized_text2 = tokenizer.tokenize(marked_text2)
        indexed_tokens2 = tokenizer.convert_tokens_to_ids(tokenized_text2)
        segment_ids2 = [1] * len(tokenized_text2)
        token_tensor2 = torch.tensor([indexed_tokens2])
        segments_tensors2 = torch.tensor([segment_ids2])
        model = BertModel.from_pretrained('bert-base-uncased')
        with torch.no_grad():
          encoded_layers, _ = model(token_tensor2,segments_tensors2)
        layer_i = 0

        batch_i = 0

        token_i = 0


        token_i = 5
        layer_i = 5
        vec = encoded_layers[layer_i][batch_i][token_i]
        token_vecs2 = encoded_layers[11][0]

        sentence_embedding2 = torch.mean(token_vecs2, dim=0)
        
        #finding the cosine similarity of the sentence embeddings of the pair of sentences
        sen1_sen3_similarity =  cosine_similarity(sentence_embedding.reshape(1,-1),sentence_embedding2.reshape(1,-1))
        list3.append(sen1_sen3_similarity)
    average1 = Average(list3) #finding the average for calculating single similarity index
    list4.append(average1)    





[              1031
cl            18856
##s            2015
]              1033
smoke          5610
read           3191
educate       16957
weed          17901
illegal        6206
lil           13451
know           2113
your           2115
##e            2063
scar          11228
stone          2962
high           2152
pay            3477
chan           9212
##c            2278
baby           3336
believe        2903
[              1031
sep           19802
]              1033
Number of layers: 12
Number of batches: 1
Number of tokens: 24
Number of hidden units: 768
[              1031
cl            18856
##s            2015
]              1033
gunn          22079
##a            2050
weed          17901
lil           13451
tonight        3892
drop           4530
different      2367
now            2085
##play        13068
##ing          2075
red            2417
albums         4042
notes          3964
new            2047
flu           19857
##kota        27380
conditional   18462
form     

In [0]:
#exporting the values of similarity indices to csv files
import csv
with open('/content/drive/My Drive/tweets1.csv', 'w') as f:
  fi = csv.writer(f, quoting = csv.QUOTE_ALL)
  fi.writerow(list4)
!cat /content/drive/My\ Drive/tweets1.csv
files.download('/content/drive/My Drive/tweets1.csv')